In [150]:
import geopandas as gpd
import json as json
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import numpy as np
import folium
import ast

In [139]:
PARISHES_TO_REMOVE = [
    'União das freguesias de Gondomar (São Cosme), Valbom e Jovim',
    'São Félix Da Marinha',
    'União das freguesias de Fânzeres e São Pedro da Cova',
    'Oliveira Do Douro',
    'Vilar De Andorinho',
    'União das freguesias de Gulpilhares e Valadares',
    'Ermesinde'
]
pd.options.display.max_rows = None

In [130]:
routes_df = pd.read_csv('../data/routes.csv')

In [131]:
routes_df['od'] = routes_df.apply(lambda x: (x['orig'], x['dest'], x['mode']), axis = 1)
# od_duration = dict(zip([x for x in routes_df['od']],[x for x in routes_df['duration']]))
# od_distance_route = dict(zip([x for x in routes_df['od']],[x for x in routes_df['distance_route']]))

In [133]:
# Filter routes:
print('Before removing points: ' + str(len(routes_df)))
routes_df = routes_df[[x not in PARISHES_TO_REMOVE for x in routes_df['orig_parish']]]
routes_df = routes_df[[x not in PARISHES_TO_REMOVE for x in routes_df['dest_parish']]]
print('After removing points: ' + str(len(routes_df)))

Before removing points: 37296
After removing points: 22968


In [134]:
od_df = routes_df.groupby(['orig','dest','orig_coordinates','dest_coordinates','orig_address','dest_address','distance']).size().reset_index(name='count')

In [152]:
od_df['orig_coordinates'] = [ast.literal_eval(x) for x in od_df['orig_coordinates']]
od_df['dest_coordinates'] = [ast.literal_eval(x) for x in od_df['dest_coordinates']]

In [135]:
od_df['duration_driving'] = od_df.apply(lambda x: od_duration[(x['orig'], x['dest'], 'driving')], axis = 1)
od_df['duration_transit'] = od_df.apply(lambda x: od_duration[(x['orig'], x['dest'], 'transit')], axis = 1)
od_df['duration_walking'] = od_df.apply(lambda x: od_duration[(x['orig'], x['dest'], 'walking')], axis = 1)
od_df['duration_driving_minutes'] = [x/60 for x in od_df['duration_driving']]
od_df['duration_transit_minutes'] = [x/60 for x in od_df['duration_transit']]
od_df['duration_walking_minutes'] = [x/60 for x in od_df['duration_walking']]
od_df['ratio_transit_driving'] = od_df['duration_transit']/od_df['duration_driving']
od_df['ratio_walking_transit'] = od_df['duration_walking']/od_df['duration_transit']
od_df['ratio_walking_driving'] = od_df['duration_walking']/od_df['duration_driving']

In [136]:
od_df['is_na'] = od_df['duration_transit'].isna()

In [144]:
summary_na = od_df.groupby(['orig'])[['is_na']].sum().sort_values(by = 'is_na', ascending = False)
pd.options.display.max_rows = None
summary_na

,is_na
orig,
Arcozelo1,0
Arcozelo2,0
União das freguesias de Lordelo do Ouro e Massarelos1,0
"União das freguesias de Custóias, Leça do Balio e Guifões4",0
"União das freguesias de Custóias, Leça do Balio e Guifões3",0
"União das freguesias de Custóias, Leça do Balio e Guifões2",0
"União das freguesias de Custóias, Leça do Balio e Guifões1",0
"União das freguesias de Cedofeita, Santo Ildefonso, Sé, Miragaia, São Nicolau e Vitória4",0
"União das freguesias de Cedofeita, Santo Ildefonso, Sé, Miragaia, São Nicolau e Vitória3",0


In [138]:
# Top/Worst regions
summary_origin = od_df.groupby(['orig'])[['duration_transit_minutes','duration_driving_minutes','ratio_transit_driving','ratio_walking_transit','ratio_walking_driving']].mean()
summary_origin.sort_values(by = ['ratio_transit_driving'], ascending=False)

,duration_transit_minutes,duration_driving_minutes,ratio_transit_driving,ratio_walking_transit,ratio_walking_driving
orig,,,,,
Canidelo3,86.395977,18.674138,4.689011,1.684708,7.868989
Canidelo2,64.935441,14.514559,4.594073,2.005003,9.111691
Moreira1,67.209962,15.344444,4.492234,2.286349,10.047505
Canidelo4,72.847318,16.867050,4.430998,1.947179,8.558012
Arcozelo3,80.801724,18.709195,4.428422,2.303130,9.954845
Canidelo1,61.200192,14.873755,4.185553,2.040319,8.518327
"União das freguesias de Custóias, Leça do Balio e Guifões4",53.086782,13.067816,4.154767,1.934551,8.020826
Cidade da Maia2,57.935441,14.109004,4.110166,2.348584,9.476569
União das freguesias de Mafamude e Vilar do Paraíso2,70.031801,17.264943,4.105505,1.997026,8.046048


In [143]:
# Worst connections
od_df.sort_values(by = ['ratio_transit_driving'], ascending=False)

,orig,dest,orig_coordinates,dest_coordinates,orig_address,dest_address,distance,count,duration_driving,duration_transit,duration_walking,duration_driving_minutes,duration_transit_minutes,duration_walking_minutes,ratio_transit_driving,ratio_walking_transit,ratio_walking_driving,is_na
5291,"União das freguesias de Custóias, Leça do Bali...",União das freguesias de Matosinhos e Leça da P...,"(41.19974189417791, -8.659609114523972)","(41.1972772450835, -8.684049820470046)","R. Monte dos Pipos 263, 4460-058 Guifões, Port...","Hipódromo, 4450 Leça da Palmeira, Portugal",2068.333058,3,142.0,1593.0,3952.0,2.366667,26.550000,65.866667,11.218310,2.480854,27.830986,False
2380,Ermesinde4,Moreira1,"(41.22293333397243, -8.547232299892004)","(41.240731287684746, -8.643618596645142)","Marginal, 4445-416 Ermesinde, Portugal","Calçada de Real 224, 4470 Moreira, Portugal",8319.225251,3,552.0,5404.0,7987.0,9.200000,90.066667,133.116667,9.789855,1.477979,14.469203,False
5918,União das freguesias de Mafamude e Vilar do Pa...,Arcozelo3,"(41.10102963697738, -8.604233330936188)","(41.06330781338246, -8.631475513509688)","R. de Montemar 443, 4430-147 Vila Nova de Gaia...","R. Rio Vale 946, 4410-396 Arcozelo, Portugal",4773.872777,3,605.0,5191.0,4462.0,10.083333,86.516667,74.366667,8.580165,0.859565,7.375207,False
241,Arcozelo3,União das freguesias de Mafamude e Vilar do Pa...,"(41.06330781338246, -8.631475513509688)","(41.10102963697738, -8.604233330936188)","R. Rio Vale 946, 4410-396 Arcozelo, Portugal","R. de Montemar 443, 4430-147 Vila Nova de Gaia...",4773.872777,3,620.0,5146.0,4951.0,10.333333,85.766667,82.516667,8.300000,0.962106,7.985484,False
7601,Águas Santas4,Moreira1,"(41.22414172719695, -8.567966581308948)","(41.240731287684746, -8.643618596645142)","R. Central de Ardegães 289, 4425-046 Águas San...","Calçada de Real 224, 4470 Moreira, Portugal",6604.758543,3,642.0,5096.0,6597.0,10.700000,84.933333,109.950000,7.937695,1.294545,10.275701,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5034,"União das freguesias de Cedofeita, Santo Ildef...",União das freguesias de Santa Marinha e São Pe...,"(41.15751036684415, -8.610382993486974)","(41.13520053195008, -8.608968114200811)","Rua do Paraíso 278, 4000-376 Porto, Portugal","R. Particular João Félix 318, 4430-148 Vila No...",2480.502912,3,900.0,842.0,2063.0,15.000000,14.033333,34.383333,0.935556,2.450119,2.292222,False
5102,"União das freguesias de Cedofeita, Santo Ildef...","União das freguesias de Cedofeita, Santo Ildef...","(41.148728956741216, -8.619852181331009)","(41.14566192616174, -8.609307087895221)","R. Prof. Jaime Rios de Sousa 51, 4050-381 Port...","R. da Madeira 156, 4000-542 Porto, Portugal",948.505228,3,999.0,925.0,944.0,16.650000,15.416667,15.733333,0.925926,1.020541,0.944945,False
4098,Ramalde4,Bonfim2,"(41.16704354371804, -8.635387884020716)","(41.144844056598046, -8.59155121328617)","R. Direita de Francos 1102, 4250-192 Porto, Po...","R. do Barão de Nova Sintra 329, 4300 Porto, Po...",4429.102503,3,1210.0,1106.0,3924.0,20.166667,18.433333,65.400000,0.914050,3.547920,3.242975,False
4947,"União das freguesias de Cedofeita, Santo Ildef...",União das freguesias de Santa Marinha e São Pe...,"(41.14566192616174, -8.609307087895221)","(41.13520053195008, -8.608968114200811)","R. da Madeira 156, 4000-542 Porto, Portugal","R. Particular João Félix 318, 4430-148 Vila No...",1162.155622,3,726.0,603.0,1224.0,12.100000,10.050000,20.400000,0.830579,2.029851,1.685950,False


In [141]:
# Best connections
od_df.sort_values(by = ['ratio_transit_driving'], ascending=True)
od_df[od_df['duration_walking_minutes']>45].sort_values(by = ['ratio_transit_driving'], ascending=True)

In [154]:
worst_df = od_df.sort_values(by = ['ratio_transit_driving'], ascending=False).head(100)

# create a folium map centered on a specific location
m = folium.Map(location=[41.157977, -8.629093], zoom_start=12)

# loop through each row in od_df
for index, row in worst_df.iterrows():
    # get the origin and destination coordinates
    orig_coords = row['orig_coordinates']
    dest_coords = row['dest_coordinates']
    
    # create a polyline between the origin and destination coordinates
    polyline = folium.PolyLine(locations=[orig_coords, dest_coords], color='red', weight=2)
    
    # add the polyline to the map
    polyline.add_to(m)

# display the map
m


In [157]:
best_df = od_df.sort_values(by = ['ratio_transit_driving'], ascending=True).head(100)

# create a folium map centered on a specific location
m = folium.Map(location=[41.157977, -8.629093], zoom_start=12)

# loop through each row in od_df
for index, row in best_df.iterrows():
    # get the origin and destination coordinates
    orig_coords = row['orig_coordinates']
    dest_coords = row['dest_coordinates']
    
    # create a polyline between the origin and destination coordinates
    polyline = folium.PolyLine(locations=[orig_coords, dest_coords], color='green', weight=2)
    
    # add the polyline to the map
    polyline.add_to(m)

# display the map
m


In [155]:
worst_df

,orig,dest,orig_coordinates,dest_coordinates,orig_address,dest_address,distance,count,duration_driving,duration_transit,duration_walking,duration_driving_minutes,duration_transit_minutes,duration_walking_minutes,ratio_transit_driving,ratio_walking_transit,ratio_walking_driving,is_na
5291,"União das freguesias de Custóias, Leça do Bali...",União das freguesias de Matosinhos e Leça da P...,"(41.19974189417791, -8.659609114523972)","(41.1972772450835, -8.684049820470046)","R. Monte dos Pipos 263, 4460-058 Guifões, Port...","Hipódromo, 4450 Leça da Palmeira, Portugal",2068.333058,3,142.0,1593.0,3952.0,2.366667,26.550000,65.866667,11.218310,2.480854,27.830986,False
2380,Ermesinde4,Moreira1,"(41.22293333397243, -8.547232299892004)","(41.240731287684746, -8.643618596645142)","Marginal, 4445-416 Ermesinde, Portugal","Calçada de Real 224, 4470 Moreira, Portugal",8319.225251,3,552.0,5404.0,7987.0,9.200000,90.066667,133.116667,9.789855,1.477979,14.469203,False
5918,União das freguesias de Mafamude e Vilar do Pa...,Arcozelo3,"(41.10102963697738, -8.604233330936188)","(41.06330781338246, -8.631475513509688)","R. de Montemar 443, 4430-147 Vila Nova de Gaia...","R. Rio Vale 946, 4410-396 Arcozelo, Portugal",4773.872777,3,605.0,5191.0,4462.0,10.083333,86.516667,74.366667,8.580165,0.859565,7.375207,False
241,Arcozelo3,União das freguesias de Mafamude e Vilar do Pa...,"(41.06330781338246, -8.631475513509688)","(41.10102963697738, -8.604233330936188)","R. Rio Vale 946, 4410-396 Arcozelo, Portugal","R. de Montemar 443, 4430-147 Vila Nova de Gaia...",4773.872777,3,620.0,5146.0,4951.0,10.333333,85.766667,82.516667,8.300000,0.962106,7.985484,False
7601,Águas Santas4,Moreira1,"(41.22414172719695, -8.567966581308948)","(41.240731287684746, -8.643618596645142)","R. Central de Ardegães 289, 4425-046 Águas San...","Calçada de Real 224, 4470 Moreira, Portugal",6604.758543,3,642.0,5096.0,6597.0,10.700000,84.933333,109.950000,7.937695,1.294545,10.275701,False
1544,Canidelo2,União das freguesias de Lordelo do Ouro e Mass...,"(41.12081391488806, -8.645036993064176)","(41.153428782746815, -8.649991662980792)","R. da Qt. dos Moinhos 101, 4400 Canidelo, Port...","Tv. Condominhas 124, 4150-226 Porto, Portugal",3645.905331,3,515.0,3924.0,6858.0,8.583333,65.400000,114.300000,7.619417,1.747706,13.316505,False
1669,Canidelo4,Canidelo1,"(41.13630074113908, -8.662187082237002)","(41.134219960605, -8.644999995973095)","Av. Agustina Bessa Luís 747, 4400 Vila Nova de...","Rua de Bustes 889, 4400 Canidelo, Portugal",1461.463458,3,166.0,1247.0,1407.0,2.766667,20.783333,23.450000,7.512048,1.128308,8.475904,False
1985,Cidade da Maia3,União das freguesias de Matosinhos e Leça da P...,"(41.2166775298441, -8.60579066593011)","(41.1972772450835, -8.684049820470046)","R. Dr. António José de Almeida 868, 4470 Gueif...","Hipódromo, 4450 Leça da Palmeira, Portugal",6908.282370,3,600.0,4504.0,8565.0,10.000000,75.066667,142.750000,7.506667,1.901643,14.275000,False
1973,Cidade da Maia3,"União das freguesias de Custóias, Leça do Bali...","(41.2166775298441, -8.60579066593011)","(41.19974189417791, -8.659609114523972)","R. Dr. António José de Almeida 868, 4470 Gueif...","R. Monte dos Pipos 263, 4460-058 Guifões, Port...",4889.933448,3,459.0,3390.0,4813.0,7.650000,56.500000,80.216667,7.385621,1.419764,10.485839,False
2855,Moreira1,União das freguesias de Matosinhos e Leça da P...,"(41.240731287684746, -8.643618596645142)","(41.1972772450835, -8.684049820470046)","Calçada de Real 224, 4470 Moreira, Portugal","Hipódromo, 4450 Leça da Palmeira, Portugal",5897.829418,3,506.0,3737.0,6978.0,8.433333,62.283333,116.300000,7.385375,1.867273,13.790514,False
